In [13]:
import boto3
import pandas as pd

# Test AWS connection
s3 = boto3.client('s3')

# List your bucket contents
try:
    response = s3.list_objects_v2(Bucket='transperth-data-shandilya')
    print("✅ S3 Connection successful!")
    
    if 'Contents' in response:
        print("Files in your bucket:")
        for obj in response['Contents']:
            print(f"  📄 {obj['Key']} ({obj['Size']} bytes)")
    else:
        print("Bucket is empty")
        
except Exception as e:
    print(f"❌ Error: {e}")

✅ S3 Connection successful!
Files in your bucket:
  📄 Unsaved/2025/08/01/3384ee98-9d89-4764-91bc-7b734c13eb37.csv (2010 bytes)
  📄 Unsaved/2025/08/01/3384ee98-9d89-4764-91bc-7b734c13eb37.csv.metadata (702 bytes)
  📄 Unsaved/2025/08/01/6618b27b-5c4f-4644-9130-5d3c1561c39b.csv (2010 bytes)
  📄 Unsaved/2025/08/01/6618b27b-5c4f-4644-9130-5d3c1561c39b.csv.metadata (702 bytes)
  📄 processed/ (0 bytes)
  📄 raw/ (0 bytes)
  📄 raw/bus_saftey.csv (3390536 bytes)


In [15]:
#Step 2: Load CSV from S3
print("\n2️⃣ Loading CSV from S3...")
try:
    # Get the CSV file
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=FILE_KEY)
    csv_content = obj['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    
    print(f"✅ Successfully loaded {len(df)} rows from S3")
    print(f"📊 Columns: {list(df.columns)}")
    
except Exception as e:
    print(f"❌ Error loading CSV: {e}")
    print("Make sure bus_saftey.csv is uploaded to s3://transperth-data-shandilya/raw/")
    exit()

# Step 3: Data Cleaning
print("\n3️⃣ Cleaning data...")



2️⃣ Loading CSV from S3...
✅ Successfully loaded 23158 rows from S3
📊 Columns: ['Year', 'Date Of Incident', 'Route', 'Operator', 'Group Name', 'Bus Garage', 'Borough', 'Injury Result Description', 'Incident Event Type', 'Victim Category', 'Victims Sex', 'Victims Age']

3️⃣ Cleaning data...


In [16]:
# Step 3: Data Cleaning
print("\n3️⃣ Cleaning data...")

# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print(f"🧹 Cleaned column names: {list(df.columns)}")

# Define columns we want to use
desired_cols = [
    'route', 
    'borough', 
    'injury_result_description', 
    'incident_event_type', 
    'victim_category', 
    'victims_sex', 
    'victims_age'
]

# Check which columns actually exist
available_cols = [col for col in desired_cols if col in df.columns]
missing_cols = [col for col in desired_cols if col not in df.columns]

if missing_cols:
    print(f"⚠️  Missing columns: {missing_cols}")
    print(f"✅ Using available columns: {available_cols}")

if not available_cols:
    print("❌ No required columns found! Please check your CSV structure.")
    exit()

# Use available columns and remove missing data
df_clean = df[available_cols].dropna()
print(f"📈 Rows after cleaning: {len(df_clean)} (removed {len(df) - len(df_clean)} rows with missing data)")

# Step 4: Create Target Variable
print("\n4️⃣ Creating target variable...")

if 'injury_result_description' in df_clean.columns:
    # Create high_risk target based on injury severity
    df_clean['high_risk'] = df_clean['injury_result_description'].apply(
        lambda x: 1 if any(keyword in str(x).lower() for keyword in ['serious', 'hospital', 'fatal', 'severe']) else 0
    )
    
    high_risk_count = df_clean['high_risk'].sum()
    total_count = len(df_clean)
    
    print(f"🎯 High risk cases: {high_risk_count} out of {total_count} ({high_risk_count/total_count:.2%})")
    
    # Show some examples
    print("\n📋 Sample injury descriptions:")
    print(df_clean['injury_result_description'].value_counts().head())
    
else:
    print("❌ No 'injury_result_description' column found - cannot create target variable")
    exit()


3️⃣ Cleaning data...
🧹 Cleaned column names: ['year', 'date_of_incident', 'route', 'operator', 'group_name', 'bus_garage', 'borough', 'injury_result_description', 'incident_event_type', 'victim_category', 'victims_sex', 'victims_age']
📈 Rows after cleaning: 23158 (removed 0 rows with missing data)

4️⃣ Creating target variable...
🎯 High risk cases: 5822 out of 23158 (25.14%)

📋 Sample injury descriptions:
Injuries treated on scene                                          17336
Taken to Hospital ? Reported Serious Injury or Severity Unknown     2994
Reported Minor Injury - Treated at Hospital                         2786
Fatal                                                                 42
Name: injury_result_description, dtype: int64


In [17]:
# Step 5: Feature Engineering
print("\n5️⃣ Preparing features...")

# Get categorical columns (exclude target and description)
categorical_cols = [col for col in available_cols if col not in ['injury_result_description']]
print(f"🏷️  Categorical columns: {categorical_cols}")

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df_clean, columns=categorical_cols, prefix=categorical_cols)

# Separate features and target
X = df_encoded.drop(['injury_result_description', 'high_risk'], axis=1)
y = df_encoded['high_risk']

print(f"🔢 Final feature count: {X.shape[1]} columns")
print(f"📊 Dataset shape: {X.shape[0]} rows × {X.shape[1]} features")

# Step 6: Train Model
print("\n6️⃣ Training machine learning model...")

if y.sum() == 0:
    print("❌ No high-risk cases found! Cannot train model.")
    print("💡 Try adjusting the high-risk criteria in step 4")
    exit()

# Handle class imbalance with class weights
classes = np.unique(y)
class_weights = compute_class_weight('balanced', classes=classes, y=y)
class_weight_dict = dict(zip(classes, class_weights))

print(f"⚖️  Class distribution: {dict(y.value_counts())}")
print(f"⚖️  Applied class weights: {class_weight_dict}")

# Train Random Forest with balanced weights
model = RandomForestClassifier(
    random_state=42, 
    n_estimators=100,
    class_weight='balanced',
    max_depth=10
)

model.fit(X, y)
print("✅ Model training completed!")

# Step 7: Feature Importance Analysis
print("\n7️⃣ Analyzing feature importance...")

# Get feature importance
feature_importance = pd.Series(model.feature_importances_, index=X.columns)
top_features = feature_importance.nlargest(15)

print("🏆 Top 10 Risk Factors:")
for i, (feature, importance) in enumerate(top_features.head(10).items(), 1):
    print(f"{i:2d}. {feature}: {importance:.4f}")


5️⃣ Preparing features...
🏷️  Categorical columns: ['route', 'borough', 'incident_event_type', 'victim_category', 'victims_sex', 'victims_age']
🔢 Final feature count: 682 columns
📊 Dataset shape: 23158 rows × 682 features

6️⃣ Training machine learning model...
⚖️  Class distribution: {0: 17336, 1: 5822}
⚖️  Applied class weights: {0: 0.6679164743885556, 1: 1.988835451734799}
✅ Model training completed!

7️⃣ Analyzing feature importance...
🏆 Top 10 Risk Factors:
 1. victims_age_Elderly: 0.1301
 2. incident_event_type_Collision Incident: 0.1245
 3. incident_event_type_Personal Injury: 0.1046
 4. victims_age_Unknown: 0.0883
 5. victim_category_Pedestrian: 0.0757
 6. victim_category_Passenger: 0.0425
 7. victims_sex_Unknown: 0.0327
 8. victims_age_Adult: 0.0234
 9. incident_event_type_Onboard Injuries: 0.0175
10. victims_age_Child: 0.0151


In [18]:

# Step 9: Model Performance Summary
print("\n9️⃣ Model Summary:")
print("="*40)

# Basic model info
train_accuracy = model.score(X, y)
print(f"📈 Training Accuracy: {train_accuracy:.3f}")
print(f"🌳 Number of Trees: {model.n_estimators}")
print(f"🔍 Features Used: {len(X.columns)}")

# Feature importance stats
print(f"\n🎯 Feature Importance Stats:")
print(f"   • Most important: {top_features.index[0]} ({top_features.iloc[0]:.4f})")
print(f"   • Average importance: {feature_importance.mean():.4f}")
print(f"   • Features >1% importance: {(feature_importance > 0.01).sum()}")

# Data summary
print(f"\n📊 Dataset Summary:")
print(f"   • Total incidents: {len(df_clean):,}")
print(f"   • High-risk incidents: {y.sum():,} ({y.mean():.2%})")
print(f"   • Low-risk incidents: {(y == 0).sum():,} ({(y == 0).mean():.2%})")

print("\n🎉 Analysis completed successfully!")
print("="*50)

# Optional: Save results to CSV
feature_importance_df = pd.DataFrame({
    'Feature': top_features.index,
    'Importance': top_features.values
})

feature_importance_df.to_csv('feature_importance_results.csv', index=False)
print("💾 Feature importance saved to 'feature_importance_results.csv'")

print("\n💡 Next steps:")
print("1. Review the top risk factors in the chart")
print("2. Focus safety initiatives on high-importance features")
print("3. Collect more data for better model accuracy")
print("4. Consider additional features like weather, time of day, etc.")



9️⃣ Model Summary:
📈 Training Accuracy: 0.696
🌳 Number of Trees: 100
🔍 Features Used: 682

🎯 Feature Importance Stats:
   • Most important: victims_age_Elderly (0.1301)
   • Average importance: 0.0015
   • Features >1% importance: 14

📊 Dataset Summary:
   • Total incidents: 23,158
   • High-risk incidents: 5,822 (25.14%)
   • Low-risk incidents: 17,336 (74.86%)

🎉 Analysis completed successfully!
💾 Feature importance saved to 'feature_importance_results.csv'

💡 Next steps:
1. Review the top risk factors in the chart
2. Focus safety initiatives on high-importance features
3. Collect more data for better model accuracy
4. Consider additional features like weather, time of day, etc.


# code for getting tableau ready

In [19]:
import pandas as pd
import boto3
from io import StringIO

# Load your bus safety data from S3
BUCKET_NAME = 'transperth-data-shandilya'
FILE_KEY = 'raw/bus_saftey.csv'

print("🚀 Preparing data for Tableau Dashboard...")
print("="*50)

# Load data from S3
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=BUCKET_NAME, Key=FILE_KEY)
csv_content = obj['Body'].read().decode('utf-8')
df = pd.read_csv(StringIO(csv_content))

print(f"📊 Loaded {len(df)} records")

# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Create enhanced dataset for Tableau
tableau_data = df.copy()

# 1. Create Risk Categories
def categorize_injury_severity(injury_desc):
    injury_str = str(injury_desc).lower()
    if any(word in injury_str for word in ['fatal', 'death', 'died']):
        return 'Fatal'
    elif any(word in injury_str for word in ['serious', 'severe', 'critical']):
        return 'Serious'
    elif any(word in injury_str for word in ['hospital', 'admitted', 'emergency']):
        return 'Hospital Required'
    elif any(word in injury_str for word in ['minor', 'slight', 'bruise', 'scratch']):
        return 'Minor'
    else:
        return 'Unknown'

if 'injury_result_description' in tableau_data.columns:
    tableau_data['injury_severity'] = tableau_data['injury_result_description'].apply(categorize_injury_severity)
    tableau_data['high_risk'] = tableau_data['injury_severity'].apply(
        lambda x: 'High Risk' if x in ['Fatal', 'Serious', 'Hospital Required'] else 'Low Risk'
    )

# 2. Create Age Groups
def categorize_age(age):
    age_str = str(age).lower()
    if any(word in age_str for word in ['child', 'infant', 'toddler', 'kid']):
        return 'Child (0-12)'
    elif any(word in age_str for word in ['teen', 'adolescent', '13', '14', '15', '16', '17']):
        return 'Teen (13-17)' 
    elif any(word in age_str for word in ['young', 'adult', '18', '19', '20', '30']):
        return 'Young Adult (18-35)'
    elif any(word in age_str for word in ['middle', 'adult', '40', '50']):
        return 'Middle Age (36-55)'
    elif any(word in age_str for word in ['senior', 'elderly', 'old', '60', '70', '80']):
        return 'Senior (55+)'
    else:
        return 'Unknown Age'

if 'victims_age' in tableau_data.columns:
    tableau_data['age_group'] = tableau_data['victims_age'].apply(categorize_age)

# 3. Create Time Categories (if date/time columns exist)
# Add day of week, month, hour analysis if timestamp columns are available
date_columns = [col for col in tableau_data.columns if any(word in col.lower() for word in ['date', 'time', 'when'])]
if date_columns:
    print(f"📅 Found date columns: {date_columns}")
    # You can add time-based analysis here based on your actual date columns

# 4. Create Location Risk Scores
if 'borough' in tableau_data.columns:
    # Calculate risk score by borough
    borough_risk = tableau_data.groupby('borough').agg({
        'high_risk': lambda x: (x == 'High Risk').mean(),
        'injury_result_description': 'count'
    }).reset_index()
    borough_risk.columns = ['borough', 'risk_score', 'total_incidents']
    borough_risk['risk_level'] = pd.cut(borough_risk['risk_score'], 
                                       bins=[0, 0.2, 0.4, 0.6, 1.0], 
                                       labels=['Low', 'Medium', 'High', 'Very High'])
    
    # Merge back to main data
    tableau_data = tableau_data.merge(borough_risk[['borough', 'risk_score', 'risk_level']], 
                                     on='borough', how='left')

# 5. Create Incident Type Categories
if 'incident_event_type' in tableau_data.columns:
    def categorize_incident_type(incident):
        incident_str = str(incident).lower()
        if any(word in incident_str for word in ['collision', 'crash', 'impact']):
            return 'Vehicle Collision'
        elif any(word in incident_str for word in ['slip', 'fall', 'trip']):
            return 'Slip/Fall'
        elif any(word in incident_str for word in ['door', 'boarding', 'alighting']):
            return 'Boarding/Alighting'
        elif any(word in incident_str for word in ['sudden', 'brake', 'jerk', 'stop']):
            return 'Sudden Movement'
        else:
            return 'Other'
    
    tableau_data['incident_category'] = tableau_data['incident_event_type'].apply(categorize_incident_type)

# 6. Create KPI Columns for Dashboard
total_incidents = len(tableau_data)
if 'high_risk' in tableau_data.columns:
    high_risk_incidents = (tableau_data['high_risk'] == 'High Risk').sum()
    tableau_data['total_incidents'] = total_incidents
    tableau_data['high_risk_count'] = high_risk_incidents
    tableau_data['risk_percentage'] = (high_risk_incidents / total_incidents) * 100

# 7. Save prepared data for Tableau
# Save to local CSV
output_file = 'bus_safety_tableau_ready.csv'
tableau_data.to_csv(output_file, index=False)
print(f"✅ Tableau-ready data saved to '{output_file}'")

# Also save to S3 processed folder
try:
    s3.put_object(
        Bucket=BUCKET_NAME,
        Key='processed/bus_safety_tableau_ready.csv',
        Body=tableau_data.to_csv(index=False)
    )
    print("✅ Data also saved to S3: s3://transperth-data-shandilya/processed/")
except Exception as e:
    print(f"⚠️  Could not save to S3: {e}")

# 8. Create Summary Statistics for Dashboard
print("\n📈 Dashboard Data Summary:")
print("="*30)
print(f"Total Incidents: {total_incidents:,}")

if 'high_risk' in tableau_data.columns:
    print(f"High Risk Incidents: {high_risk_incidents:,}")
    print(f"Risk Rate: {(high_risk_incidents/total_incidents)*100:.1f}%")

if 'age_group' in tableau_data.columns:
    print(f"\n👥 Age Groups:")
    for age, count in tableau_data['age_group'].value_counts().head().items():
        print(f"   {age}: {count}")

if 'borough' in tableau_data.columns:
    print(f"\n🏘️  Top Risk Areas:")
    risk_areas = tableau_data.groupby('borough')['high_risk'].apply(lambda x: (x == 'High Risk').mean()).sort_values(ascending=False)
    for area, risk in risk_areas.head().items():
        print(f"   {area}: {risk:.1%}")

if 'incident_category' in tableau_data.columns:
    print(f"\n🚨 Incident Types:")
    for incident, count in tableau_data['incident_category'].value_counts().head().items():
        print(f"   {incident}: {count}")

print(f"\n📊 Columns created for Tableau: {len(tableau_data.columns)}")
print("Key columns:", [col for col in tableau_data.columns if col in [
    'injury_severity', 'high_risk', 'age_group', 'risk_score', 'risk_level', 
    'incident_category', 'total_incidents', 'risk_percentage'
]])

print("\n🎯 Ready for Tableau! Use the 'bus_safety_tableau_ready.csv' file")
print("="*50)

🚀 Preparing data for Tableau Dashboard...
📊 Loaded 23158 records
📅 Found date columns: ['date_of_incident']
✅ Tableau-ready data saved to 'bus_safety_tableau_ready.csv'
✅ Data also saved to S3: s3://transperth-data-shandilya/processed/

📈 Dashboard Data Summary:
Total Incidents: 23,158
High Risk Incidents: 5,822
Risk Rate: 25.1%

👥 Age Groups:
   Young Adult (18-35): 10754
   Unknown Age: 7454
   Senior (55+): 2769
   Child (0-12): 2181

🏘️  Top Risk Areas:
   Kensington & Chelsea: 32.0%
   Westminster: 29.8%
   None London Borough: 28.7%
   City of London: 28.4%
   Sutton: 28.0%

🚨 Incident Types:
   Other: 12011
   Slip/Fall: 6981
   Vehicle Collision: 4166

📊 Columns created for Tableau: 21
Key columns: ['injury_severity', 'high_risk', 'age_group', 'risk_score', 'risk_level', 'incident_category', 'total_incidents', 'risk_percentage']

🎯 Ready for Tableau! Use the 'bus_safety_tableau_ready.csv' file
